# Extra Code for Emerson's Homeownership Project

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ipumspy.readers as readers # For census data (retrived from IPUMS)

In [ ]:
cols = [
    "enterprise_flag",
    "record_number",
    "state_fips",
    "msa_code",
    "county_fips",
    "census_tract",
    "tract_pct_minority",
    "tract_median_income",
    "local_median_income",
    "tract_income_ratio",
    "borrower_income",
    "area_median_income",
    "borrower_income_ratio",
    "upb_acquisition",
    "purpose",
    "fed_guarantee",
    "num_borrowers",
    "first_time_homebuyer",
    "borrower_race",
    "borrower_ethnicity",
    "coborrower_race",
    "coborrower_ethnicity",
    "borrower_gender",
    "coborrower_gender",
    "borrwer_age",
    "coborrower_age",
    "occupancy_code",
    "rate_spread",
    "hoepa_status",
    "property_type",
    "lien_status",
    "borrower_over_62",
    "co_borrower_over_62",
    "ltv",
    "date_of_note",
    "term_at_org",
    "num_of_units",
    "interest_rate_at_org",
    "note_amount",
    "preapproval",
    "application_channel",
    "aus_name",
    "borrower_credit_score_model",
    "coborrower_credit_score_model",
    "dti_ratio",
    "discount_pts",
    "intro_period",
    "land_prop_interest",
    "property_value",
    "rural_tract",
    "lower_ms_delta",
    "middle_appalachia",
    "persistent_poverty",
    "concentrated_poverty",
    "high_opportunity",
    "qoz_tract",
]
dropcols = [
    "enterprise_flag",
    "record_number",
    "borrower_income",
    "upb_acquisition",
    "purpose",
    "fed_guarantee",
    "rate_spread",
    "hoepa_status",
    "property_type",
    "lien_status",
    "borrower_over_62",
    "co_borrower_over_62",
    "ltv",
    "term_at_org",
    "num_of_units",
    "interest_rate_at_org",
    "preapproval",
    "application_channel",
    "aus_name",
    "borrower_credit_score_model",
    "coborrower_credit_score_model",
    "discount_pts",
    "intro_period",
    "land_prop_interest",
    "rural_tract",
    "lower_ms_delta",
    "middle_appalachia",
    "persistent_poverty",
    "concentrated_poverty",
    "high_opportunity",
    "qoz_tract",
]
cols_pre2018 = [
    "enterprise_flag",
    "record_number",
    "state_fips",
    "msa_code",
    "county_fips",
    "census_tract",
    "tract_pct_minority",
    "tract_median_income",
    "local_median_income",
    "tract_income_ratio",
    "borrower_income",
    "area_median_income",
    "borrower_income_ratio",
    "upb_acquisition",
    "purpose",
    "fed_guarantee",
    "num_borrowers",
    "first_time_homebuyer",
    "borrower_race",
    "borrower_ethnicity",
    "coborrower_race",
    "coborrower_ethnicity",
    "borrower_gender",
    "coborrower_gender",
    "borrwer_age",
    "coborrower_age",
    "occupancy_code",
    "rate_spread",
    "hoepa_status",
    "property_type",
    "lien_status",
]
dropcols_pre2018 = [
    "enterprise_flag",
    "record_number",
    "borrower_income",
    "upb_acquisition",
    "purpose",
    "fed_guarantee",
    "rate_spread",
    "hoepa_status",
    "property_type",
    "lien_status",
]

In [ ]:
with open("data/columns.txt", "w") as f:
    f.write("cols = " + str(cols) + "\n")
    f.write("dropcols = " + str(dropcols) + "\n")
    f.write("cols_pre2018 = " + str(cols_pre2018) + "\n")
    f.write("dropcols_pre2018 = " + str(dropcols_pre2018) + "\n")

In addition, we want to have some dictionaries to decode data. For example, in the `loans` dataframe, each metropolitan area is given by its "MSA code", which means nothing to us. Rather, we want to know the MSA name.

In [ ]:
msa_codes = pd.read_csv('data/MSA_2023.txt', sep='\s+=\s+', header=None, engine='python', names=['msa_code', 'msa_name'])
loans = loans.merge(msa_codes, on='msa_code', how='left')

Next, we start loading in data. Below is the code to load in the data from all years. This takes approximately 26 and a half minutes to run.

In [ ]:
loans_list = []

# Loans from 2018-2023 (new system)
for year in range(2018, 2023):
    files = [f"data/sf/fhlmc_sf{year}c_loans.txt", f"data/sf/fnma_sf{year}c_loans.txt"]
    loans_year = pd.concat(
        [pd.read_csv(file, sep=r"\s+", header=None, names=cols).drop(columns=dropcols) for file in files],
        ignore_index=True
    )
    loans_year["year"] = year
    loans_list.append(loans_year)
    print(f"Processed {year}")
print("First data done.")

# Loans from 2008-2017 (old system)
for year in range(2008, 2018):
    files = [f"data/sf/fhlmc_sf{year}c_loans.txt", f"data/sf/fnma_sf{year}c_loans.txt"]
    loans_year = pd.concat(
        [pd.read_csv(file, sep=r"\s+", header=None, names=cols_pre2018).drop(columns=dropcols_pre2018) for file in files],
        ignore_index=True
    )
    loans_year["year"] = year
    loans_list.append(loans_year)
    print(f"Processed {year}")

loans = pd.concat(loans_list, ignore_index=True)

# Saving
loans.to_parquet("data/loans_data.parquet", index=False)  

print("Data saved successfully.")

In [ ]:
# loans = pd.read_parquet("data/loans_data.parquet")